In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
import os
import numpy as np
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [15]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths within your Google Drive
drive_path = '/content/drive/MyDrive/'


# Path to your training CSV (update the path as needed)
# The original path was likely incorrect.
# Assuming your CSV is within a folder called 'dataset_splits' in your Google Drive:
train_csv_path = os.path.join(drive_path, 'dataset_splits', 'train', 'train.csv')

# Load the training data
train_df = pd.read_csv(train_csv_path)

# Rename columns for clarity: assuming 'v2' is text and 'v1' is label
train_df = train_df.rename(columns={"sms": "text", "label": "label"})

# Use a subset (e.g., 200 examples) for prompt engineering
train_subset = train_df.sample(n=200, random_state=69420)

# Map numeric labels to strings for zero-shot evaluation.
# We will use candidate labels "yes" (spam) and "no" (not spam)
def map_label(label):
    return "spam" if label == 1 else "not spam"

train_subset['mapped_label'] = train_subset['label'].apply(map_label)

# Extract texts and true labels
texts_subset = train_subset["text"].tolist()
true_labels = train_subset["mapped_label"].tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# New candidate labels
candidate_labels = ["spam", "not spam"]

# New prompt templates to try
template_options = [
    "This message is {}.",
    "Classify this text as {}.",
    "The text is {}."
]

In [17]:
# Model 1: facebook/bart-large-mnli
model1_name = "facebook/bart-large-mnli"
classifier_model1 = pipeline("zero-shot-classification", model=model1_name, device=0)

# Model 2: valhalla/distilbart-mnli-12-3
model2_name = "valhalla/distilbart-mnli-12-3"
classifier_model2 = pipeline("zero-shot-classification", model=model2_name, device=0)


Device set to use cuda:0
Device set to use cuda:0


In [19]:
def get_predictions(classifier, texts, candidate_labels, hypothesis_template):
    preds = []
    for text in texts:
        result = classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
        # The predicted label is the first in the "labels" list
        preds.append(result["labels"][0])
    return preds

def evaluate_predictions(preds, true_labels):
    acc = accuracy_score(true_labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average="binary", pos_label="spam")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Evaluate each combination on the training subset
results = {}

for model_name, classifier in [("Model1_BART", classifier_model1), ("Model2_DistilBART", classifier_model2)]:
    for template in template_options:
        preds = get_predictions(classifier, texts_subset, candidate_labels, hypothesis_template=template)
        metrics = evaluate_predictions(preds, true_labels)  # true_labels should also be updated accordingly
        results[f"{model_name} with template: '{template}'"] = metrics

# Display the evaluation results
for key, metrics in results.items():
    print(key)
    print(metrics)
    print()

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model1_BART with template: 'This message is {}.'
{'accuracy': 0.875, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

Model1_BART with template: 'Classify this text as {}.'
{'accuracy': 0.88, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

Model1_BART with template: 'The text is {}.'
{'accuracy': 0.865, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

Model2_DistilBART with template: 'This message is {}.'
{'accuracy': 0.88, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

Model2_DistilBART with template: 'Classify this text as {}.'
{'accuracy': 0.88, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

Model2_DistilBART with template: 'The text is {}.'
{'accuracy': 0.88, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Load the test data (similar to the training data)
# Update the path to include your Google Drive path if the file is there
test_csv_path = os.path.join(drive_path, 'dataset_splits', 'test', 'test.csv')
test_df = pd.read_csv(test_csv_path)
test_df = test_df.rename(columns={"sms": "text", "label": "label"})
test_df['mapped_label'] = test_df['label'].apply(map_label)

# Extract texts and true labels from the test set
texts_test = test_df["text"].tolist()
true_labels_test = test_df["mapped_label"].tolist()

# Function to evaluate a zero-shot model on the test set using the chosen template
def evaluate_on_test(classifier, hypothesis_template):
    preds = get_predictions(classifier, texts_test, candidate_labels, hypothesis_template)
    metrics = evaluate_predictions(preds, true_labels_test)
    return metrics

# Assume Template 1 was selected for both models; adjust if needed based on Cell 6.
test_results_model1 = evaluate_on_test(classifier_model1, template1)
test_results_model2 = evaluate_on_test(classifier_model2, template1)

print("Test Set Evaluation - Model 1 (facebook/bart-large-mnli) with Template 1:")
print(test_results_model1)
print("\nTest Set Evaluation - Model 2 (valhalla/distilbart-mnli-12-3) with Template 1:")
print(test_results_model2)


Test Set Evaluation - Model 1 (facebook/bart-large-mnli) with Template 1:
{'accuracy': 0.7784753363228699, 'precision': 0.15, 'recall': 0.14093959731543623, 'f1': 0.1453287197231834}

Test Set Evaluation - Model 2 (valhalla/distilbart-mnli-12-3) with Template 1:
{'accuracy': 0.831390134529148, 'precision': 0.046511627906976744, 'recall': 0.013422818791946308, 'f1': 0.020833333333333332}


In [23]:
import os
import pandas as pd

# Make sure you have mounted Google Drive first:
# from google.colab import drive
# drive.mount('/content/drive')

# Specify the absolute path in your Google Drive where you want to save the results
drive_save_path = "/content/drive/MyDrive/ZeroShotResults"

# Create the folder if it doesn't already exist
os.makedirs(drive_save_path, exist_ok=True)

# Assume these variables exist from your zero-shot experiment:
# texts_test: List of test texts
# preds: List of predictions from the zero-shot pipeline
# true_labels_test: List of true labels for the test set

# For demonstration purposes, here's an example dummy data:
# texts_test = ["Example text 1", "Example text 2"]
# preds = ["spam", "not spam"]
# true_labels_test = ["spam", "spam"]

# Generate predictions for the entire test set using your chosen model and template
preds = get_predictions(classifier_model1, texts_test, candidate_labels, template_options[0]) # Assuming template_options[0] is your desired template

# Create a DataFrame with your results
# Now preds will have the same length as texts_test and true_labels_test
results_df = pd.DataFrame({
    "text": texts_test,
    "prediction": preds,
    "true_label": true_labels_test
})

# Define the full path for saving the CSV file
csv_path = os.path.join(drive_save_path, "zero_shot_predictions.csv")

# Save the DataFrame as a CSV file
results_df.to_csv(csv_path, index=False)
print("Predictions saved to", csv_path)


Predictions saved to /content/drive/MyDrive/ZeroShotResults/zero_shot_predictions.csv
